# 🌊 基流分割教程：河流的银行账户

## 欢迎来到水文学的世界！

各位大一新生，欢迎！今天我们将探索一个有趣的水文学概念：**基流分割**（Baseflow Separation）。

### 📖 故事时间：河流就像银行账户

想象一下，河流就像你的**银行账户**：

- 💰 **基流（Baseflow）**：这是你的**工资收入**——稳定、持续、可预测。它来自地下水的缓慢释放，就像每个月固定到账的工资。
  
- 🌧️ **快速径流（Quickflow）**：这是你的**奖金或红包**——突然出现、数额大、但不持久。它来自降雨后的地表径流，就像过年收到的红包，来得快去得也快。

- 🏞️ **总径流（Total Streamflow）**：这是你账户的**总余额** = 工资 + 奖金。

**我们的任务**：给定河流的总流量（总余额），我们要把它分解成基流（工资）和快速径流（奖金）两部分。这就是**基流分割**！

---

## 🎯 学习目标

通过本教程，你将学会：
1. 如何加载和可视化河流流量数据
2. 使用 **Eckhardt 滤波器**进行基流分割
3. 通过交互式滑块探索参数对结果的影响
4. 理解基流分割背后的简单数学原理

## 📦 第一步：准备工具箱

首先，我们需要导入一些 Python 库。这些库就像你的工具箱，帮助我们完成任务。

In [ ]:
# 数据处理
import numpy as np
import pandas as pd
import warnings

# 可视化
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
# 改进字体配置，支持中文和特殊字符
plt.rcParams['font.sans-serif'] = ['SimHei', 'Microsoft YaHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False
# 使用 mathtext 来处理上标/下标
plt.rcParams['mathtext.default'] = 'regular'

# 禁用字体警告（superscript 3 缺失是已知问题）
warnings.filterwarnings('ignore', message='.*Glyph.*missing from font.*')

# 交互式小部件
import ipywidgets as widgets
from ipywidgets import interact, FloatSlider

# 基流分割库
import sys
sys.path.insert(0, r'c:\Users\ASUS\OneDrive\SCI\Github\Baseflow_Seperation\src')
from baseflow.synthetic_data import generate_streamflow
from baseflow.methods import Eckhardt
from baseflow.param_estimate import recession_coefficient
from baseflow.comparision import strict_baseflow
from baseflow.utils import exist_ice
from baseflow.methods.lh_core import lh_filter

print("✅ 工具箱准备完毕！")

## 🌊 第二步：生成河流数据

为了便于学习，我们将生成一年的**模拟河流流量数据**。这些数据包含：
- 稳定的基流（来自地下水）
- 间歇性的降雨事件（产生快速径流）

最棒的是：我们知道**真实的基流**是什么样的，这样可以验证我们的分割方法是否准确！

In [ ]:
# 生成一年的河流数据
Q_total, Q_baseflow_true, Q_quickflow = generate_streamflow(
    n_days=365,           # 365天
    base_flow=15.0,       # 平均基流 15 m³/s
    seasonal_amplitude=5.0,  # 季节波动幅度
    n_storm_events=25,    # 25次降雨事件
    storm_intensity=50.0, # 平均降雨强度
    bfi=0.65,             # 基流指数 (Baseflow Index) = 65%
    random_seed=42        # 随机种子（保证结果可重复）
)

# 创建日期索引
dates = pd.date_range('2020-01-01', periods=365, freq='D')

# 转换为 pandas Series
flow_series = pd.Series(Q_total, index=dates, name='流量 (m3/s)')
baseflow_true_series = pd.Series(Q_baseflow_true, index=dates, name='真实基流 (m3/s)')

print(f"📊 数据生成成功！")
print(f"   - 数据长度: {len(flow_series)} 天")
print(f"   - 平均流量: {flow_series.mean():.2f} m3/s")
print(f"   - 最大流量: {flow_series.max():.2f} m3/s")
print(f"   - 最小流量: {flow_series.min():.2f} m3/s")
print(f"   - 真实基流指数 BFI: {Q_baseflow_true.sum() / Q_total.sum():.2%}")

## 📈 第三步：可视化河流流量

让我们画出河流流量随时间的变化。你能看到：
- 📉 **平缓的低流量期**：这主要是基流
- 📈 **突然的流量峰值**：这是降雨事件产生的快速径流

In [ ]:
# 创建图表
fig, ax = plt.subplots(figsize=(14, 5), dpi=100)

# 绘制总流量（蓝色）
ax.plot(dates, Q_total, color='#2E86AB', linewidth=1.5, label='总流量 (观测值)', zorder=2)

# 绘制真实基流（橙色，半透明）
ax.plot(dates, Q_baseflow_true, color='#F77F00', linewidth=2, label='真实基流', zorder=3)
ax.fill_between(dates, 0, Q_baseflow_true, color='#F77F00', alpha=0.2, label='基流区域', zorder=1)

# 美化图表
ax.set_xlabel('日期', fontsize=12, fontweight='bold')
ax.set_ylabel('流量 (m³/s)', fontsize=12, fontweight='bold')
ax.set_title('河流流量时间序列：总流量 vs 真实基流', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', fontsize=10, framealpha=0.9)
ax.grid(True, alpha=0.3, linestyle='--')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("💡 观察：")
print("   - 蓝色线：河流的总流量（我们观测到的）")
print("   - 橙色区域：真实的基流（实际应用中我们不知道它是什么，但这里为了教学展示）")
print("   - 尖峰：降雨事件产生的快速径流")

## 🔬 第四步：应用 Eckhardt 滤波器

### 什么是 Eckhardt 滤波器？

Eckhardt 滤波器是一种**数字滤波方法**，它通过数学公式从总流量中提取基流。这个方法由德国科学家 Klaus Eckhardt 在 2005 年提出。

### 📐 数学原理（简化版）

Eckhardt 滤波器的核心公式是：

$$
b_{t+1} = \frac{(1 - BFI_{max}) \cdot \alpha \cdot b_t + (1 - \alpha) \cdot BFI_{max} \cdot Q_{t+1}}{1 - \alpha \cdot BFI_{max}}
$$

**符号说明**：
- $b_t$：第 $t$ 天的基流
- $Q_t$：第 $t$ 天的总流量（观测值）
- $\alpha$：**消退系数**（recession coefficient），通常在 0.9 到 0.995 之间
  - 📌 物理意义：描述地下水释放的速度。值越接近 1，地下水释放越慢（基流越稳定）
- $BFI_{max}$：**最大基流指数**，表示基流占总流量的最大比例（通常 0.5~0.8）

**约束条件**：
$$
b_t \leq Q_t \quad \text{(基流不能超过总流量)}
$$

### 🛠️ 实施步骤

1. **估计消退系数 α**：从数据中自动计算
2. **设定 BFI_max**：我们将使用 0.8（可以根据流域特征调整）
3. **应用滤波器**：计算每一天的基流

In [ ]:
# 步骤 1: 估计消退系数 α
Q_array = flow_series.values
ice_mask = exist_ice(dates, None)  # 假设没有冰冻期
strict = strict_baseflow(Q_array, ice_mask)
alpha = recession_coefficient(Q_array, strict)

print(f"🔍 参数估计：")
print(f"   - 消退系数 α = {alpha:.4f}")
print(f"   - 物理意义：地下水每天保留约 {alpha*100:.1f}% 的水量")

# 步骤 2: 计算初始基流（使用 LH 滤波器）
b_LH = lh_filter(Q_array, beta=0.925)

# 步骤 3: 应用 Eckhardt 滤波器
BFImax = 0.80  # 最大基流指数
baseflow_eckhardt = Eckhardt(Q_array, b_LH, alpha, BFImax)

# 计算基流指数
calculated_bfi = baseflow_eckhardt.sum() / Q_array.sum()

print(f"\n✅ 基流分割完成！")
print(f"   - 计算得到的基流指数 BFI = {calculated_bfi:.2%}")
print(f"   - 真实基流指数 = {Q_baseflow_true.sum() / Q_total.sum():.2%}")
print(f"   - 误差 = {abs(calculated_bfi - Q_baseflow_true.sum() / Q_total.sum()):.2%}")

## 🎨 第五步：对比结果

让我们把 Eckhardt 方法计算出的基流和真实基流放在一起对比。

In [ ]:
# 创建对比图
fig, ax = plt.subplots(figsize=(14, 6), dpi=100)

# 绘制总流量（浅蓝色，半透明）
ax.plot(dates, Q_total, color='#2E86AB', linewidth=1.5, alpha=0.6, label='总流量 (观测值)', zorder=1)

# 绘制真实基流（橙色）
ax.plot(dates, Q_baseflow_true, color='#F77F00', linewidth=2, label='真实基流', linestyle='--', zorder=3)

# 绘制 Eckhardt 计算的基流（绿色）
ax.plot(dates, baseflow_eckhardt, color='#06A77D', linewidth=2.5, label='Eckhardt 基流', zorder=4)
ax.fill_between(dates, 0, baseflow_eckhardt, color='#06A77D', alpha=0.25, label='Eckhardt 基流区域', zorder=2)

# 美化图表
ax.set_xlabel('日期', fontsize=12, fontweight='bold')
ax.set_ylabel('流量 (m³/s)', fontsize=12, fontweight='bold')
ax.set_title('基流分割结果对比：Eckhardt 方法 vs 真实基流', fontsize=14, fontweight='bold', pad=20)
ax.legend(loc='upper right', fontsize=10, framealpha=0.9)
ax.grid(True, alpha=0.3, linestyle='--')
ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()

print("💡 结果解读：")
print("   - 绿色线：Eckhardt 方法计算的基流")
print("   - 橙色虚线：真实基流（用于对比）")
print("   - 两条线非常接近，说明 Eckhardt 方法效果很好！")

## 🎛️ 第六步：交互式探索 - 参数的影响

现在最有趣的部分来了！我们将创建一个**交互式滑块**，让你可以调整参数 $\alpha$（消退系数），实时观察基流线如何变化。

### 🧪 实验任务

拖动滑块，观察：
- 当 $\alpha$ **接近 1**（例如 0.995）时会发生什么？
- 当 $\alpha$ **较小**（例如 0.90）时会发生什么？

思考：为什么会有这样的变化？

In [ ]:
# 定义交互式绘图函数
def plot_baseflow_interactive(alpha_value):
    """
    根据给定的 alpha 值绘制基流分割结果
    """
    # 使用新的 alpha 值计算基流
    baseflow_new = Eckhardt(Q_array, b_LH, alpha_value, BFImax)
    bfi_new = baseflow_new.sum() / Q_array.sum()
    
    # 创建图表
    fig, ax = plt.subplots(figsize=(14, 6), dpi=100)
    
    # 绘制总流量
    ax.plot(dates, Q_total, color='#2E86AB', linewidth=1.5, alpha=0.5, label='总流量', zorder=1)
    
    # 绘制真实基流（参考）
    ax.plot(dates, Q_baseflow_true, color='#F77F00', linewidth=1.5, 
            label='真实基流（参考）', linestyle=':', alpha=0.7, zorder=2)
    
    # 绘制当前 alpha 计算的基流
    ax.plot(dates, baseflow_new, color='#D62828', linewidth=2.5, label=f'基流 (α={alpha_value:.3f})', zorder=3)
    ax.fill_between(dates, 0, baseflow_new, color='#D62828', alpha=0.25, zorder=0)
    
    # 添加参数信息文本框
    textstr = f'α = {alpha_value:.3f}\nBFI = {bfi_new:.2%}\n真实 BFI = {Q_baseflow_true.sum() / Q_total.sum():.2%}'
    props = dict(boxstyle='round', facecolor='wheat', alpha=0.8)
    ax.text(0.02, 0.98, textstr, transform=ax.transAxes, fontsize=11,
            verticalalignment='top', bbox=props, family='monospace')
    
    # 美化图表
    ax.set_xlabel('日期', fontsize=12, fontweight='bold')
    ax.set_ylabel('流量 (m³/s)', fontsize=12, fontweight='bold')
    ax.set_title(f'交互式基流分割：调整消退系数 α', fontsize=14, fontweight='bold', pad=20)
    ax.legend(loc='upper right', fontsize=10, framealpha=0.9)
    ax.grid(True, alpha=0.3, linestyle='--')
    ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m'))
    ax.xaxis.set_major_locator(mdates.MonthLocator(interval=2))
    plt.xticks(rotation=45)
    
    plt.tight_layout()
    plt.show()

# 创建交互式滑块
interact(
    plot_baseflow_interactive,
    alpha_value=FloatSlider(
        value=alpha,  # 初始值为自动估计的值
        min=0.900,
        max=0.995,
        step=0.005,
        description='消退系数 α:',
        style={'description_width': '120px'},
        layout=widgets.Layout(width='600px'),
        readout_format='.3f'
    )
);

## 🤔 思考题

通过上面的交互式实验，回答以下问题：

1. **当 α 增大时（接近 1）**，基流线有什么变化？为什么？
   - 提示：想想"银行账户"的比喻——如果钱存得越久（α 大），支出速度会怎样？

2. **当 α 减小时（接近 0.9）**，基流线有什么变化？
   - 提示：如果钱花得很快，账户余额会如何波动？

3. **什么样的 α 值能让计算的基流最接近真实基流？**
   - 对比红色线和橙色点线

4. **基流指数（BFI）的物理意义是什么？**
   - BFI = 基流总量 / 总流量
   - BFI 越大意味着什么？

## 📊 第七步：性能评估

让我们用数字来评估 Eckhardt 方法的准确性。

In [ ]:
# 计算误差指标
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

mae = mean_absolute_error(Q_baseflow_true, baseflow_eckhardt)
rmse = np.sqrt(mean_squared_error(Q_baseflow_true, baseflow_eckhardt))
r2 = r2_score(Q_baseflow_true, baseflow_eckhardt)
relative_error = np.abs(baseflow_eckhardt - Q_baseflow_true).mean() / Q_baseflow_true.mean()

print("📈 性能评估指标：")
print(f"   - 平均绝对误差 (MAE): {mae:.3f} m³/s")
print(f"   - 均方根误差 (RMSE): {rmse:.3f} m³/s")
print(f"   - 决定系数 (R²): {r2:.4f}")
print(f"   - 相对误差: {relative_error:.2%}")
print(f"\n💡 解释：")
print(f"   - R² = {r2:.4f} 表示模型解释了 {r2*100:.1f}% 的变异")
print(f"   - 平均每天的误差约为 {mae:.2f} m³/s")
print(f"   - 相对误差仅 {relative_error:.1%}，非常准确！")

## 🎓 总结

恭喜你完成了基流分割教程！让我们回顾一下学到的内容：

### ✨ 关键概念

1. **基流 vs 快速径流**
   - 基流：来自地下水，稳定持久（工资）
   - 快速径流:来自降雨,突然短暂(奖金)

2. **Eckhardt 滤波器**
   - 使用递推公式从总流量中提取基流
   - 关键参数：α (消退系数) 和 BFI_max (最大基流指数)

3. **参数的影响**
   - α 越大 → 基流越平滑、越稳定
   - α 越小 → 基流波动越大、响应越快

### 🚀 下一步

你可以尝试：
- 修改 `BFImax` 参数，观察对结果的影响
- 生成不同特征的河流数据（改变 `base_flow`, `n_storm_events` 等）
- 探索其他基流分割方法（Chapman, LH, Boughton 等）
- 应用到真实的河流流量数据

### 📚 延伸阅读

- Eckhardt, K. (2005). How to construct recursive digital filters for baseflow separation. *Hydrological Processes*, 19(2), 507-515.
- Nathan, R. J., & McMahon, T. A. (1990). Evaluation of automated techniques for base flow and recession analyses. *Water Resources Research*, 26(7), 1465-1473.

---

**感谢使用本教程！祝你在水文学的学习道路上越走越远！** 🌊📖

## 🔧 附录：使用真实数据

如果你有真实的河流流量数据（CSV 文件），可以使用以下代码加载：

In [ ]:
# 示例：加载真实数据
# df = pd.read_csv('your_data.csv', index_col=0, parse_dates=True)
# flow_series = df['flow']  # 假设流量列名为 'flow'
# 
# # 然后重复上面的步骤
# Q_array = flow_series.values
# alpha = recession_coefficient(Q_array, strict_baseflow(Q_array, exist_ice(flow_series.index, None)))
# b_LH = lh_filter(Q_array, beta=0.925)
# baseflow = Eckhardt(Q_array, b_LH, alpha, BFImax=0.80)

print("📝 提示：将上面的注释取消，替换为你的数据文件路径即可！")